Vou montar aqui um pipeline mais robusto, mais próximo do produtivo, focando em lgbm

# Definições iniciais

## Imports

In [19]:
# Sklearn
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, RandomizedSearchCV, train_test_split, StratifiedKFold,cross_val_score
from sklearn.model_selection import ParameterGrid, ParameterSampler
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

# Hyperopt
from hyperopt import fmin, hp, tpe, rand, Trials, space_eval, STATUS_OK, anneal
from hyperopt.pyll import scope as ho_scope
from hyperopt.pyll.stochastic import sample as ho_sample
from functools import partial

# Category encoders
from category_encoders.hashing import HashingEncoder
from category_encoders.sum_coding import SumEncoder
from category_encoders.woe import WOEEncoder
from category_encoders.target_encoder import TargetEncoder
from category_encoders.one_hot import OneHotEncoder
from category_encoders.binary import BinaryEncoder

# Plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
%matplotlib inline

# Files
# Files
from tempfile import mkdtemp
from shutil import rmtree
from joblib import Memory
import joblib
import os


## Configurando random pra deixar reprodutível

In [4]:
import random
random.seed(42)
np.random.seed(42)
random_global = 42

# Funções úteis

## Função pra submeter os resultados e salvar os arquivos necessários pra replicar

In [5]:
class SaveModel(object):
    def __init__(self, folder_to_save, data_train=None, data_val=None, data_test=None, model=None, str_readme=None, submission_file='submission.csv'):
        self.folder_to_save = folder_to_save
        self.data_train = data_train
        self.data_val = data_val
        self.data_test = data_test
        self.model = model
        self.str_readme = str_readme
        self.submission_file = submission_file
    
    def save_model(self):
#     Create folder if not exists:
        try:
            os.makedirs(self.folder_to_save)
        except:
            pass

    #     Salva os dados usados no treino
        if self.data_train is not None:
            joblib.dump(self.data_train, self.folder_to_save+'/train_data')


    #     Salva dados usados na validação
        if self.data_val is not None:
            joblib.dump(self.data_test, self.folder_to_save+'/validation_data')

    #     Salva dados usados no teste
        if self.data_test is not None:
            joblib.dump(self.data_test, self.folder_to_save+'/test_data')

    #     Salva modelo 
        if self.model is not None:
            joblib.dump(self.model, self.folder_to_save+'/model')   

    #     Arquivo README (é o que vai escrito pro commit)
        with open(self.folder_to_save+'/README.txt', "w") as text_file:
            text_file.write(self.str_readme)
            
#         Salva os predictions
        

    def commit_kaggle(self):
        predictions = self.model.predict(self.data_test)
        submission = pd.DataFrame({'PassengerId':self.data_test['passengerid'],'Survived':predictions})
        submission.to_csv(self.folder_to_save+'/'+self.submission_file,index=False)
#         print(f"kaggle competitions submit -c titanic -f submission.csv -m \"{self.str_readme}\"")
#         !! f"kaggle competitions submit -c titanic -f {self.folder_to_save+'/'+self.submission_file} -m \"{self.str_readme}\""
        if os.system(f"kaggle competitions submit -c titanic -f {self.folder_to_save+'/'+self.submission_file} -m \"{self.str_readme}\"") != 0:
            print('Erro submetendo o arquivo no kaggle!')
            
        print(f"kaggle competitions submit -c titanic -f {self.folder_to_save+'/'+self.submission_file} -m \"{self.str_readme}\"")
    
        

## Custom transformers

In [32]:
from sklearn.base import BaseEstimator, TransformerMixin
from functools import reduce

class TransformToDF(BaseEstimator, TransformerMixin):
    '''Wrapper para usar transformers do sklearn mas retorna um dataframe Pandas. Projetei para usar com transformers que não
    mudam o número de colunas na saída.'''
    
    def __init__(self, sklearn_transformer, return_df=True):
        self.sklearn_transformer = sklearn_transformer
        self.return_df = return_df
    
    def fit(self, X, y=None):
        if isinstance(X, pd.DataFrame):
            self.col_names = X.columns.values.tolist()
            
        self.sklearn_transformer.fit(X, y=None)
        return self
        
    def transform(self, X, y=None):
    # assumes X is a DataFrame
        if self.return_df:
            return pd.DataFrame(self.sklearn_transformer.transform(X[self.col_names]), index=X.index, columns=self.col_names)
        else:
            return self.sklearn_transformer.transform(X)
        
class DFFeatureUnion(BaseEstimator,TransformerMixin):
    # FeatureUnion but for pandas DataFrames

    def __init__(self, transformer_list):
        self.transformer_list = transformer_list

    def fit(self, X, y=None):
        for (name, t) in self.transformer_list:
            t.fit(X, y)
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        Xts = [t.transform(X) for _, t in self.transformer_list]
        Xunion = reduce(lambda X1, X2: pd.merge(X1, X2, left_index=True, right_index=True), Xts)
        return Xunion
    
class ColumnExtractor(BaseEstimator,TransformerMixin):

    def __init__(self, cols, return_df=True):
        self.cols = cols
        self.return_df = return_df

    def fit(self, X, y=None):
        # stateless transformer
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        if self.return_df:
            return X[self.cols]
        else:
            return X[self.cols].values

# Carrega dados


In [10]:
df_train_val = pd.read_csv("./data/train.csv")
df_test = pd.read_csv("./data/test.csv")
df_train_val.columns = [x.lower() for x in df_train_val.columns]
df_test.columns = [x.lower() for x in df_test.columns]

print(f'Print shape df_train_val: {df_train_val.shape}')
print(f'Print shape df_test: {df_test.shape}')


Print shape df_train_val: (891, 12)
Print shape df_test: (418, 11)


# Criação de algumas features novas

## Tamanho de família

In [11]:
full_data = [df_train_val, df_test]
for dataset in full_data:
    dataset['family_size'] = 1 + dataset['parch'] + dataset['sibsp']
    if 'survived' in dataset.columns:
        display(dataset.fillna(-999).groupby('family_size')['survived'].agg(['size', 'mean']).reset_index())
    else:
        display(dataset.fillna(-999).groupby('family_size').size())

,family_size,size,mean
0,1,537,0.303538
1,2,161,0.552795
2,3,102,0.578431
3,4,29,0.724138
4,5,15,0.200000
5,6,22,0.136364
6,7,12,0.333333
7,8,6,0.000000
8,11,7,0.000000


family_size
1     253
2      74
3      57
4      14
5       7
6       3
7       4
8       2
11      4
dtype: int64

Aparentemente depois de 4 familiares, talvez seja bom juntar depois

## Extrai o título

In [12]:
full_data = [df_train_val, df_test]
for dataset in full_data:
    dataset['title'] = dataset['name'].str.findall('([A-Z][a-z]+)\.').map(lambda x: x[0])
    if 'survived' in dataset.columns:
        display(dataset.fillna(-999).groupby('title')['survived'].agg(['size', 'mean']).reset_index().sort_values(by='mean', ascending=False))
    else:
        display(dataset.fillna(-999).groupby('title').size())
        

,title,size,mean
16,Sir,1,1.000000
2,Countess,1,1.000000
14,Ms,1,1.000000
11,Mme,1,1.000000
6,Lady,1,1.000000
10,Mlle,2,1.000000
13,Mrs,125,0.792000
9,Miss,182,0.697802
8,Master,40,0.575000
1,Col,2,0.500000


title
Col         2
Dona        1
Dr          1
Master     21
Miss       78
Mr        240
Mrs        72
Ms          1
Rev         2
dtype: int64

## Acha aspas ou parênteses no nome

In [13]:
full_data = [df_train_val, df_test]
for dataset in full_data:
    dataset['name_aspas'] = dataset['name'].str.findall('\"').map(lambda x: len(x) >= 1 ).astype('int')
    dataset['name_parenteses'] = dataset['name'].str.findall('\(').map(lambda x: len(x) >= 1 ).astype('int')
    if 'survived' in dataset.columns:
        display(dataset.fillna(-999).groupby('name_aspas')['survived'].agg(['size', 'mean']).reset_index().sort_values(by='mean', ascending=False))
        display(dataset.fillna(-999).groupby('name_parenteses')['survived'].agg(['size', 'mean']).reset_index().sort_values(by='mean', ascending=False))
    else:
        display(dataset.fillna(-999).groupby('name_aspas').size())
        display(dataset.fillna(-999).groupby('name_parenteses').size())
        

,name_aspas,size,mean
1,1,53,0.716981
0,0,838,0.362768


,name_parenteses,size,mean
1,1,143,0.769231
0,0,748,0.310160


name_aspas
0    396
1     22
dtype: int64

name_parenteses
0    340
1     78
dtype: int64

# Separa as amostras para treino e validação

In [14]:
df_train, df_val = train_test_split(df_train_val, test_size = 100, shuffle=True, stratify=df_train_val['survived'], random_state=random_global)
print(f'Print shape df_train: {df_train.shape}')
print(f'Print shape df_val: {df_val.shape}')

Print shape df_train: (791, 16)
Print shape df_val: (100, 16)


# Definição do pipeline básico a ser otimizado

In [15]:
num_cols = ['age', 'sibsp', 'parch', 'fare', 'family_size']
cat_cols = ['sex', 'embarked', 'title', 'name_aspas', 'name_parenteses', 'pclass']

In [16]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_global)

In [40]:
from lightgbm import LGBMClassifier
hp_space_lgb ={
        'num_missing_strategy': hp.choice('num_missing_strategy', ['mean', 'median', 'constant']),
        'categ_encoder': hp.choice('categ_encoder', [WOEEncoder(cols=cat_cols, return_df=True, hp.choice())
                                                 ,OneHotEncoder(cols=cat_cols, use_cat_names=True, return_df=True)
                                                 ,TargetEncoder(cols=cat_cols, return_df=True)])
}

In [47]:
for train_index, test_index in kf.split(df_train, df_train['survived']):
    print('train', train_index.shape)
    print('test', test_index.shape)

train (632,)
test (159,)
train (632,)
test (159,)
train (633,)
test (158,)
train (633,)
test (158,)
train (634,)
test (157,)
